In [ ]:
!pip install pip==21.3.1
!pip install mediapipe-model-maker
!pip install mediapipe opencv-python
!pip install pandas
!pip install matplotlib
!pip install openpyxl

In [ ]:
!mkdir output
!mkdir output/frames

In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def getX(landmarks, xValues):
  for idx, landmark in enumerate(landmarks):
    if mp_pose.PoseLandmark(idx).name == "LEFT_HIP":
      xValues.append(landmark.x)

In [ ]:
video_path = './input.mp4'

In [ ]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

In [ ]:
cap = cv2.VideoCapture(video_path)

In [ ]:
frame_number = 0
xValues = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    result = pose.process(frame_rgb)

    if result.pose_landmarks:
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        getX(result.pose_landmarks.landmark, xValues)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_number += 1

    cv2.imwrite('./output/frames/frame' + str(frame_number) + '.png', frame)

cv2.destroyAllWindows()

In [ ]:
image_width_scale_meters = 10 #the real world length that corresponds to the image width, in meters

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
timeInterval = 1.0/fps

velocityValues = [0]
accelerationValues = [0]
timeValues = [0]

for i in range(1, len(xValues)):
  velocityValues.append(image_width_scale_meters * ((xValues[i] - xValues[i - 1])/timeInterval))
  accelerationValues.append((velocityValues[i] - velocityValues[i - 1])/timeInterval)
  timeValues.append(i * timeInterval)

In [ ]:
plt.plot(timeValues, velocityValues)
plt.title("Velocity vs. Time")
plt.xlabel("Time (s)")
plt.ylabel("Velocity (s)")
plt.savefig('./output/output_velocity_graph.png')

plt.plot(timeValues, accelerationValues)
plt.title("Acceleration vs. Time")
plt.xlabel("Time (s)")
plt.ylabel("Acceleration (m/s^2)")
plt.savefig('./output/output_acceleration_graph.png')

In [ ]:
df = pd.DataFrame({'time (s)': timeValues, 'velocity (m/s)': velocityValues, 'acceleration (m/s^2)': accelerationValues})
df.to_excel('./output/output_data.xlsx')